<a href="https://colab.research.google.com/github/atul-ai/prompt-engineering-class/blob/main/SimpleRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Project with a Book

## Problem
We will read a portion of the book and build chat bot on that to ask questions and also ask the bot to summarize the chapter.

## Process
Read the book from archive and extract 2 chapters as text, chop up the paragraphs and build a simple look up mechanism and pass that to a LLM to generate an answer.

In [3]:
!pip install langchain chromadb sentence-transformers langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os

# Read the text file
with open('/content/input_data/Chapter_1.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Split the text into paragraphs
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
paragraphs = text_splitter.split_text(text)

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings()

# Create and persist the vector store
persist_directory = 'db'
vectorstore = Chroma.from_texts(texts=paragraphs, embedding=embeddings, persist_directory=persist_directory)
vectorstore.persist()

print(f"Created and persisted vector store with {len(paragraphs)} paragraphs.")

<ipython-input-5-19d440f63c24>:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-5-19d440f63c24>:20: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Created and persisted vector store with 41 paragraphs.


<ipython-input-5-19d440f63c24>:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [6]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.6 MB/s eta 0:00:00


In [11]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "GROQ-KEY"


# Initialize the Groq language model
llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0)

# Create a custom prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Create the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# Function to answer questions
def answer_question(question):
    result = qa({"query": question})
    answer = result['result']
    sources = [doc.page_content for doc in result['source_documents']]
    return answer, sources

# Example usage
question = "Where did third wave of Arab conquests go?"
answer, sources = answer_question(question)
print(f"Question: {question}")
print(f"Answer: {answer}")
print("\nSources:")
for i, source in enumerate(sources, 0):
    print(f"{i}. {source[:100]}...\n")  # Print first 100 characters of each source

question = "What happened to the Pratihara and Pala kingdoms?"
answer, sources = answer_question(question)
print(f"Question: {question}")
print(f"Answer: {answer}\n")
print("Sources:")
for i, source in enumerate(sources, 0):
    print(f"{i}. {source[:100]}...\n")  # Print first 100 characters of each source

question = "What were Ghoris' intention?"
answer, sources = answer_question(question)
print(f"Question: {question}")
print(f"Answer: {answer}\n")
print("Sources:")
for i, source in enumerate(sources, 0):
    print(f"{i}. {source[:100]}...\n")  # Print first 100 characters of each source


Question: Where did third wave of Arab conquests go?
Answer: The third wave of Arab conquests went eastward and reached the borders of India and China about the middle of the eighth century.

Sources:
0. a thousand miles from the rock of Gibraltar to the
banks of the Loire; the repetition of an equal sp...

1. Asia Minor (part of Turkey), and reached the Strait of
Dardanelles. In 717 they laid siege by land a...

2. kingdoms held the invaders off for a while. Finally, an Arab
army under Muhammad Bin Qasim defeated ...

3. Chapter 1
India in Shivaji’s Times
PROMINENT MUSLIM ST ATES IN INDIA
Muhammad, the founder of Islam,...

Question: What happened to the Pratihara and Pala kingdoms?
Answer: The Pratihara and Pala kingdoms declined and fragmented into smaller states in the latter half of the 10th Century.

Sources:
0. kingdoms held the invaders off for a while. Finally, an Arab
army under Muhammad Bin Qasim defeated ...

1. 22
Coochbihar: This kingdom was situated on the western
border